# Testing how user data will be stored, processed and made ready for the javascript code running on the client to generate graphs with

In [1]:
import pandas as pd
import json
from random import randint
from time import time

## Database is loaded in memory when the server starts up


In [2]:
database = pd.DataFrame()

## Generating some random test data

In [3]:
def get_random_raw():
    """
    This is similar to the format provided by the html-form object.
    """    
    names = [f"{str(i).rjust(2, '0')}_{impairment}" for i in range(10) for impairment in range(3)]
    raw_data = [(name, randint(1, 5)) for name in names]
    # timestamp to identify user (since we're not gonna use emails)
    raw_data.insert(0, ("user", int(time()*1000)))
    return raw_data

In [4]:
get_random_raw()

[('user', 1640689715435),
 ('00_0', 5),
 ('00_1', 2),
 ('00_2', 5),
 ('01_0', 2),
 ('01_1', 5),
 ('01_2', 3),
 ('02_0', 5),
 ('02_1', 4),
 ('02_2', 2),
 ('03_0', 1),
 ('03_1', 3),
 ('03_2', 5),
 ('04_0', 4),
 ('04_1', 3),
 ('04_2', 5),
 ('05_0', 2),
 ('05_1', 4),
 ('05_2', 1),
 ('06_0', 4),
 ('06_1', 2),
 ('06_2', 1),
 ('07_0', 2),
 ('07_1', 3),
 ('07_2', 5),
 ('08_0', 1),
 ('08_1', 2),
 ('08_2', 1),
 ('09_0', 3),
 ('09_1', 2),
 ('09_2', 5)]

In [5]:
def raw_data_to_record(raw_data):
    """
    Convert raw data from the form to a record (row). 
    """
    new_record = pd.DataFrame(raw_data).T
    new_record = new_record.rename(columns=new_record.iloc[0])[1:]
    return new_record

# This function will be called at each time a new user submits their form:


In [6]:
def append_to_database(raw_data):
    """
    Append raw data from a new user to the database
    """
    global database
    new_record = raw_data_to_record(raw_data)
    database = database.append(new_record).reset_index(drop=True)

## Call it n times to append new random data from n 'users'

In [19]:
append_to_database(get_random_raw())

In [20]:
database

,user,00_0,00_1,00_2,01_0,01_1,01_2,02_0,02_1,02_2,...,06_2,07_0,07_1,07_2,08_0,08_1,08_2,09_0,09_1,09_2
0,1640689716092,2,4,4,2,5,1,5,2,4,...,1,1,4,2,1,1,2,1,2,5
1,1640689982845,5,4,1,1,3,2,5,2,1,...,1,1,3,1,1,5,5,2,2,2


## Calculating mos and mos-stddev values

In [21]:
class ImageTypes:
    ORIGINAL = "_0"
    SLIGHTLY_IMPAIRED = "_1"
    HEAVILY_IMPAIRED = "_2"

In [22]:
def get_names():
    return [col[:2] for col in database.columns if ImageTypes.ORIGINAL in col]

In [23]:
def get_mos(img_type):
    cols = [col for col in database.columns if img_type in col]
    return database[cols].mean(axis=0).to_list()

In [24]:
def get_mos_stddev(img_type):
    cols = [col for col in database.columns if img_type in col]
    return database[cols].std(axis=0).to_list()

### Eg: mos of original images:

In [25]:
get_mos(ImageTypes.ORIGINAL)

[3.5, 1.5, 5.0, 2.5, 3.5, 4.5, 3.5, 1.0, 1.0, 1.5]

### Eg: stdev of mos of heavily impaired images

In [26]:
get_mos_stddev(ImageTypes.HEAVILY_IMPAIRED)

[2.1213203435596424,
 0.7071067811865476,
 2.1213203435596424,
 0.0,
 0.0,
 0.0,
 0.0,
 0.7071067811865476,
 2.1213203435596424,
 2.1213203435596424]

# Api call functions:

In [27]:
def stats_getters_generator(stat_f):
    def getter():
        result = {}
        result["names"] = get_names()
        result["original-image-mos"] = stat_f(ImageTypes.ORIGINAL)
        result["slightly-impaired-image-mos"] = stat_f(ImageTypes.SLIGHTLY_IMPAIRED)
        result["heavily-impaired-image-mos"] = stat_f(ImageTypes.HEAVILY_IMPAIRED)
        return json.dumps(result)
    return getter

In [28]:
get_mos_json = stats_getters_generator(get_mos)
get_mos_stddev_json = stats_getters_generator(get_mos_stddev)

In [29]:
get_mos_json()

'{"names": ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09"], "original-image-mos": [3.5, 1.5, 5.0, 2.5, 3.5, 4.5, 3.5, 1.0, 1.0, 1.5], "slightly-impaired-image-mos": [4.0, 4.0, 2.0, 2.5, 3.5, 2.5, 4.5, 3.5, 3.0, 2.0], "heavily-impaired-image-mos": [2.5, 1.5, 2.5, 5.0, 2.0, 4.0, 1.0, 1.5, 3.5, 3.5]}'

In [30]:
get_mos_stddev_json()

'{"names": ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09"], "original-image-mos": [2.1213203435596424, 0.7071067811865476, 0.0, 0.7071067811865476, 0.7071067811865476, 0.7071067811865476, 2.1213203435596424, 0.0, 0.0, 0.7071067811865476], "slightly-impaired-image-mos": [0.0, 1.4142135623730951, 0.0, 2.1213203435596424, 2.1213203435596424, 2.1213203435596424, 0.7071067811865476, 0.7071067811865476, 2.8284271247461903, 0.0], "heavily-impaired-image-mos": [2.1213203435596424, 0.7071067811865476, 2.1213203435596424, 0.0, 0.0, 0.0, 0.0, 0.7071067811865476, 2.1213203435596424, 2.1213203435596424]}'